In [1]:
import scipp as sc
from scipp.plot import plot
import numpy as np
import dataconfig # run make_config.py to create this
from graphical_reduction import q1d  
import ipywidgets as w
import IPython.display as disp
import warnings

In [2]:
def load_and_return(run):
    global loaded_data
    print(f'Loading data for run {run}:')
    loaded_data[run] = sc.neutron.load(filename=f'{path}/LARMOR000{run}.nxs')
    return loaded_data[run]

def load_data(run_number_list):
    global loaded_data
    for run in run_number_list:
        if run not in loaded_data:
            load_and_return(run)
        else:
            print(f'Run {run} already loaded')

def run_reduction(sample_run_number, sample_transmission_run_number,
                  background_transmission_run_number, background_run_number):
    global loaded_data
    sample = loaded_data[sample_run_number] if sample_run_number in loaded_data else load_and_return(sample_run_number)
    sample_trans = loaded_data[sample_transmission_run_number] if sample_transmission_run_number in loaded_data else load_and_return(sample_transmission_run_number)
    background = loaded_data[background_run_number] if background_run_number in loaded_data else load_and_return(background_run_number)
    background_trans = loaded_data[background_transmission_run_number] if background_transmission_run_number in loaded_data else load_and_return(background_transmission_run_number)
    
    dtype = sample.coords['position'].dtype
    sample_pos_offset = sc.Variable(value=[0.0, 0.0, 0.30530], unit=sc.units.m, dtype=dtype)
    bench_pos_offset = sc.Variable(value=[0.0, 0.001, 0.0], unit=sc.units.m, dtype=dtype)
    for item in [sample, sample_trans, background, background_trans]:
        item.coords['sample-position'] += sample_pos_offset
        item.coords['position'] += bench_pos_offset
    
    moderator_file_path = f'{path}/{moderator_file}'
    direct_beam_file_path = f'{path}/{direct_beam_file}'
    
    global sample_q1d
    print('Reducing sample data:')
    sample_q1d = q1d(data=sample, transmission=sample_trans, 
                     l_collimation=l_collimation, r1=r1, r2=r2, dr=dr, wavelength_bins=wavelength_bins,
                     direct_beam_file_path=direct_beam_file_path, moderator_file_path=moderator_file_path,
                     wavelength_bands=None)
    
    global background_q1d
    print('Reducing background data:')
    background_q1d = q1d(data=background, transmission=background_trans,
                         l_collimation=l_collimation, r1=r1, r2=r2, dr=dr, wavelength_bins=wavelength_bins,
                         direct_beam_file_path=direct_beam_file_path, moderator_file_path=moderator_file_path,
                         wavelength_bands=None)
    
    global reduced
    reduced = sample_q1d - background_q1d

    reduced.coords['Transmission'] = sc.Variable(
        value=f'{sample_transmission_run_number}_trans_sample_0.9_13.5_unfitted')
    reduced.coords['TransmissionCan'] = sc.Variable(
        value=f'{background_transmission_run_number}_trans_can_0.9_13.5_unfitted')
    
    print('Finished Reduction')
    

In [9]:
class ReductionWidget(w.Box):
    def __init__(self):
        super().__init__()
        self.sample = w.Text(description='Sample', value='49338')
        self.sample_trans = w.Text(description='Sample trans', value='49339')
        self.background = w.Text(description='Background', value='49334')
        self.background_trans = w.Text(description='Backg... trans', value='49335')
        self.load_button = w.Button(description='Load')
        self.load_button.on_click(self._on_load_button_clicked)
        self._button = w.Button(description='Process')
        self._button.on_click(self._on_process_button_clicked)
        self.output = w.Output(width='100%', height='100%')
        self.children = [w.VBox([w.HBox([self.sample, self.load_button]), w.HBox([self.sample_trans, self._button]), self.background, self.background_trans, self.output])]
        
    def _on_process_button_clicked(self, b):
        self.output.clear_output()
        sample_run_number = self.sample.value
        sample_transmission_run_number = self.sample_trans.value
        background_run_number = self.background.value
        background_transmission_run_number = self.background_trans.value
        with self.output:
            run_reduction(sample_run_number, sample_transmission_run_number, background_transmission_run_number, background_run_number)
        
        
    def _on_load_button_clicked(self, b):
        self.output.clear_output()
        run_list = [self.sample.value, self.sample_trans.value, self.background.value, self.background_trans.value]
        with self.output:
            load_data(run_list)
            

In [4]:
class PlotWidget(w.Box):
    def __init__(self):
        super().__init__()
        options = [key for key, item in globals().items() if isinstance(item, (sc.DataArray, sc.Dataset))]
        self._data_selector = w.Combobox(placeholder='Data to plot', options=options)
        self._button = w.Button(description='Plot')
        self._button.on_click(self._on_button_clicked)
        self.output = w.Output(width='100%', height='100%')
        self.children = [w.VBox([w.HBox([self._data_selector, self._button]), self.output])]
        
    def _on_button_clicked(self, b):
        self.output.clear_output()
        data_dict = {key: globals()[key] for key in "".join(self._data_selector.value.split()).split(',')}
        ds = sc.Dataset(data_dict)
        with self.output:
            plot(ds)

In [5]:
# Set reduction settings.
path = dataconfig.data_root
direct_beam_file = 'DirectBeam_20feb_full_v3.dat'
moderator_file = 'ModeratorStdDev_TS2_SANS_LETexptl_07Aug2015.txt'
l_collimation = sc.Variable(value=5.0, unit=sc.units.m)
r2 = sc.Variable(value=4.0824829046386295/1000, unit=sc.units.m) # sample aperture radius
r1 = sc.Variable(value=14.433756729740645/1000, unit=sc.units.m) # source aperture radius
dr = sc.Variable(value=8.0/1000, unit=sc.units.m) # virtual ring width on detector
wavelength_bins = sc.Variable(
        dims=['wavelength'],
        unit=sc.units.angstrom,
        values=np.geomspace(0.9, 13.5, num=110))
loaded_data = {}

In [10]:
disp.display(ReductionWidget())

ReductionWidget(children=(VBox(children=(HBox(children=(Text(value='49338', description='Sample'), Button(desc…

In [8]:
disp.display(PlotWidget())

PlotWidget(children=(VBox(children=(HBox(children=(Combobox(value='', options=('sample_q1d', 'background_q1d',…